In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

In [2]:
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, shape
import asf_search as asf
from datetime import datetime, date, timedelta
from typing import List
from pystac_client import Client, ItemSearch
import geopandas as gpd
from rasterio.crs import CRS
import contextily as cx
import pandas as pd
from shapely.ops import unary_union
from itertools import combinations
import formatting as f
import search as s

In [27]:
from importlib import reload
reload(s)

<module 'search' from '/Users/angelac/ground-swath/Angela_notebooks/search.py'>

In [26]:
# calculate cadence
def get_cadence(results):
    
    cadence = ''
    if len(results) == 0:
        cadence = 'There is no coverage during this time'

    else:
        if len(results) == 1:
            try:
                cadence = 'Only one acquisition on ' + results.startTime[0]
            except:
                cadence = 'Only one acquisition on ' + results.start_datetime[0]

        else:
            cadence = []
            for i in range(len(results) - 1):
                try:
                    cadence.append(str(f.asfsearch2datetime(str(results.startTime[i + 1])) - f.asfsearch2datetime(str(results.startTime[i]))))
                except:
                    cadence.append(str(f.asfsearch2datetime(str(results.start_datetime[i + 1])) - f.asfsearch2datetime(str(results.start_datetime[i]))))
        
    return cadence

In [4]:
def get_coverage(sensor: List[str], aoi: Point, date: List[datetime] = None) -> List[dict]:
    """
    Sensor: choose sentinel1, sentinel2, landsat8
    AOI: enter coordinates as Polygon object
    date: leave as none if searching today, else enter time range as datetime tuple: datetime(YYYY,MM,DD)
    """
    freq = {}
    next_acq = {}
    area = {}
    
    for sensor_name in sensor:
        freq[sensor_name] = ''
        next_acq[sensor_name] = ''
        area[sensor_name] = ''
        
        if 'landsat8' in sensor_name.lower():
            results = s.hls_search('landsat8', aoi, date)
            df = f.format_results_for_hls(results)
#             print('here')
        elif 'sentinel1' in sensor_name.lower():
            results = s.asf_search(aoi, date)
            df = f.format_results_for_sent1(results)
        elif 'sentinel2' in sensor_name.lower():
            results = s.hls_search('sentinel2', aoi, date)
            df = f.format_results_for_hls(results)
        
        try:
            df = df.dissolve(by='datetime').reset_index()
        except:
            df = df.dissolve(by='startTime').reset_index()
        
        # return cadence as string or list using get_cadence
        freq[sensor_name] = get_cadence(df)
        
        # find next acquisition time, if search time is today then returns 'N/A'
        if date == None:
            next_acq[sensor_name] = 'N/A'
            
        else:
            next_acq[sensor_name] = s.acq_search(sensor_name.lower(), aoi, date[1])
        
        # find area intersection for each sensor
#         coords = [Polygon(c) for c in coords]
#         area[sensor_name] = unary_union([a.intersection(b) for a, b in combinations(coords, 2)])
        
        if len(results) == 0:
            area[sensor_name] = 0
        else:
            area[sensor_name] = df.geometry[0]
            
            if len(results) > 1:
                
                for i in range(len(df) - 1):
                    area[sensor_name] = area[sensor_name].intersection(df.geometry[i + 1])
         
    return freq, next_acq, area

### Polygons of areas of interest:

Ridgecrest coordinates: Polygon([[-117.7167, 35.5909],[-117.6322, 35.5909],[-117.6322, 35.6452],[-117.7167, 35.6452],[-117.7167, 35.5909]])

Wax lake delta: Polygon([[-91.4964, 29.4641],[-91.3849, 29.4641],[-91.3849, 29.5627],[-91.4964, 29.5627],[-91.4964, 29.4641]])

Laurentides forest in Canada: Polygon([[-75.0327, 46.0832],[-74.8823, 46.0832],[-74.8823, 46.1914],[-75.0327, 46.1914],[-75.0327, 46.0832]])

In [5]:
# create shapely and geodataframe files of areas of interest
ridgecrest = Polygon([[-117.7167, 35.5909],[-117.6322, 35.5909],[-117.6322, 35.6452],[-117.7167, 35.6452],[-117.7167, 35.5909]])
waxlake = Polygon([[-91.4964, 29.4641],[-91.3849, 29.4641],[-91.3849, 29.5627],[-91.4964, 29.5627],[-91.4964, 29.4641]])
laurentides = Polygon([[-75.0327, 46.0832],[-74.8823, 46.0832],[-74.8823, 46.1914],[-75.0327, 46.1914],[-75.0327, 46.0832]])
ridgecrest_df = f.shape2gdf(ridgecrest, 'ridgecrest')
waxlake_df = f.shape2gdf(waxlake, 'waxlake')
laurentides_df = f.shape2gdf(laurentides, 'laurentides')

In [36]:
reload(s)

<module 'search' from '/Users/angelac/ground-swath/Angela_notebooks/search.py'>

In [19]:
a = get_cadence(sent1_forest)

In [20]:
a

['11 days, 23:59:59']

In [37]:
freq3, next_acq3, area3 = get_coverage(['sentinel1','sentinel2','landsat8'],laurentides,[datetime(2022,1,1), datetime(2022,2,1)])


In [ ]:
freq3['landsat8']

### Work in progress

In [ ]:
# print(freq3['landsat8'])
# print(next_acq3['landsat8'])
# print(freq3['sentinel1'])
# print(next_acq3['sentinel1'])
print(freq3['sentinel2'])
print(next_acq3['sentinel2'])

In [ ]:
df = f.format_results_for_hls(d)

In [ ]:
df

In [23]:
df_dis = land8_forest.dissolve(by='datetime').reset_index()
df_dis.start_datetime

0    2022-01-08T15:44:18.159Z
1    2022-01-15T15:50:27.751Z
2    2022-01-24T15:44:13.503Z
3    2022-01-31T15:50:22.884Z
Name: start_datetime, dtype: object

In [ ]:
f.asfsearch2datetime(df_dis.start_datetime[0])

In [24]:
cadence = get_cadence(df_dis)

In [25]:
cadence

['7 days, 0:06:09.592000', '8 days, 23:53:45.752000', '7 days, 0:06:09.381000']

In [ ]:
world = gpd.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


In [ ]:
world.plot(figsize=(10,10))

In [ ]:
world.head()

In [ ]:
df_cont = world.dissolve(by='continent').reset_index()
df_cont.iloc[:1].plot()

In [ ]:
world.dissolve(by='continent').plot(figsize=(10,10))
# world.plot()

In [ ]:
df_1.head()

### The below cells are just for reminding what format the output results have

In [7]:
wkt = laurentides.wkt
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'processingLevel': [asf.PRODUCT_TYPE.SLC],
    'beamMode': [asf.BEAMMODE.IW],
    'start': '2022-01-01T00:00:00Z',
    'end': '2022-02-01T23:59:59Z'
}
results = asf.search(intersectsWith=wkt,**opts)
print(len(results))

2


In [8]:
sent1_forest = f.format_results_for_sent1(results)

In [9]:
type(sent1_forest.startTime[0])

pandas._libs.tslibs.timestamps.Timestamp

In [21]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
api = Client.open(f'{STAC_URL}/LPCLOUD/')
hls_collections = ['HLSL30.v2.0']
search_params = {"collections": hls_collections,
                 "bbox": [-75.0327, 46.0832, -74.8823, 46.1914], # list of xmin, ymin, xmax, ymax
                 "datetime": [datetime(2022,1,1), datetime(2022,2,1)],
                 }
search_hls = api.search(**search_params)
hls_collection = search_hls.get_all_items()
e = list(hls_collection)
# d[0].properties

In [22]:
land8_forest = f.format_results_for_hls(e)

8


In [ ]:
d[1].to_dict()

In [ ]:
df.iloc[:2].to_file('test2.geojson',driver='GeoJSON')

In [ ]:
for i in range(len(d)):
    print(d[i].properties['datetime'])

In [ ]:
shape(area1['sentinel1'])

# Visualization

In [ ]:
# returns filled-in plot or outline of dictionary of polygons (or single polygon) with world map underneath
def visual(area, outline = False):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 10)
    
    if type(area) == dict:
        
        color = ['blue','green','red']
    
        for num,frame in enumerate(area.keys()):
            df = gpd.GeoDataFrame(geometry = [area[frame]],
                                  crs = CRS.from_epsg(4326))
            df_wm = df.to_crs(epsg = 3857)
            if not outline:
                df_wm.plot(ax = ax, alpha = .3, color=color[num], legend = True)
            else:
                df_wm.boundary.plot(ax = ax, color=color[num])
            
    elif type(area) == gpd.GeoSeries:
        
        df_wm = area.to_crs(epsg = 3857)
        if not outline:
            df_wm.plot(ax = ax, alpha = .3)
        else:
            df_wm.boundary.plot(ax = ax)
        
    elif type(area) == Polygon:
        
        df = gpd.GeoDataFrame(geometry = [area],
                                  crs = CRS.from_epsg(4326))
        df_wm = df.to_crs(epsg = 3857)
        if not outline:
            df_wm.plot(ax = ax, alpha = .3)
        else:
            df_wm.boundary.plot(ax = ax)
    
    cx.add_basemap(ax, zoom = 10)

In [ ]:
# returns plot of intersection of all polygons in input dictionary
def find_overlap(area: dict, outline = False):
    
    for idx,key in enumerate(area.keys()):
        if idx == 0:
            overlap = area[key]
        else:
            overlap = overlap.intersection(area[key])
            
    visual(overlap, outline)

In [ ]:
visual(df_dis.geometry)

In [ ]:
find_overlap(area1)

In [ ]:
visual(area1['sentinel1'],outline=True)

In [ ]:
visual(area1['sentinel1'])

In [ ]:
visual(area1)

In [ ]:
visual(area2)

In [ ]:
visual(area3)

### Deprecated